# Data Preprocessing
* Below codes are identical to the first half of "Final Metrics and Graphs.ipynb"

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 150

df = pd.read_excel('AASG_Thermed_AllThicksAndConds.xlsx')
df2 = pd.read_csv('clean_new_well_data_fixed.csv')

In [6]:
def outlierDrop(df,df_toCheck,std_cut_off):
    init_len = df.shape[0]
    for i in df_toCheck:
        mean = df[i].mean()
        std = df[i].std()
        cut_off = std * std_cut_off
        lower, upper =mean - cut_off, mean + cut_off
        df = df[(df[i] < upper) & (df[i] > lower)]
    print("numbere of outliers removed: ", init_len - df.shape[0])
    return df
#df = outlierDrop(df,['CorrBHT', 'HeatFlow','MeasureDepth_m'],4)
df = outlierDrop(df,['HeatFlow'],3)
df = df[df['HeatFlow']>0]

df.reset_index(inplace=True, drop=True)

layers = df.iloc[:,52:101].values
conds = df.iloc[:,101:150].values
mult = np.multiply(layers,conds)

numbere of outliers removed:  263


# Save the XGB model as a pickle file

In [7]:
import xgboost as xgb

In [8]:
gbm = xgb.XGBRegressor(learning_rate=0.01,  
                           objective='reg:squarederror', 
                           n_estimators=2000, 
                           reg_alpha=1,
                           reg_lambda=10,
                           max_depth=10, 
                           gamma=0.1)
X_train = np.transpose(np.array([df.LatDegree,
                                df.LongDegree,
                                df.MeasureDepth_m,
                                df.SurfTemp]))
y_train = df.CorrBHT.values

In [9]:
X_train = np.concatenate((X_train, mult),axis=1)

In [10]:
gbm.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.1, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=2000, n_jobs=6, num_parallel_tree=1, random_state=0,
             reg_alpha=1, reg_lambda=10, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [13]:
import pickle
pickle.dump(gbm, open('xgbSaved.pkl','wb'))

# DNN pickle cannot seem to save from our api as below (thread error). Hence, we only provide XGB pickle file that is trained.

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
import sklearn.metrics as m
def baseline_model():
    # create model
    # Sequential model is for just building up each layer
    model = Sequential()
    # Kernel initializer sets up the distribution for the random parameters
    model.add(Dense(50, #kernel_regularizer=regularizers.l2(0.1),
                    kernel_initializer='normal', activation='relu'))
    model.add(Dense(50, #kernel_regularizer=regularizers.l2(0.1),
                    kernel_initializer='normal', activation='relu'))    
    model.add(Dense(1, kernel_initializer='normal', activation='relu'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model


model = KerasRegressor(build_fn=baseline_model, epochs=150, batch_size=50, verbose=0)
model.fit(X_train, y_train)

In [16]:
pickle.dump(model, open('dnnSaved.pkl','wb'))

TypeError: cannot pickle '_thread.RLock' object